In [1]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import ttest_1samp, ttest_ind
from scipy.stats import pearsonr


def warn(*args, **kwargs):
    pass


warnings.warn = warn

In [2]:
tick = "15m"

df_bars = pd.read_csv(
    f"../data/ohlc_{tick}_gateio.csv", index_col=0, parse_dates=True)
df_close = df_bars.filter(axis="columns", like="close")
df_close.columns = [x.split("_")[0] for x in df_close.columns]

tick = "24h"
df_close = df_close.resample(tick).last()

In [3]:
n_train_days = 120
df_ica_pred = pd.read_csv(
    f"../data/ica_arma_predictions_{tick}_{n_train_days}D.csv",
    index_col=0,
    parse_dates=True)
df_arma_pred = pd.read_csv(
    f"../data/arma_predictions_{tick}_{n_train_days}D.csv",
    index_col=0,
    parse_dates=True).reindex(df_ica_pred.index)

In [4]:
df_ret = df_close.pct_change().fillna(0).reindex(df_arma_pred.index)

In [5]:
#
# --- Mean Absolute Error ---
#
ttest_ind(
    (df_ica_pred - df_ret).abs().unstack().values, 
    (df_arma_pred - df_ret).abs().unstack().values,
    alternative="less")

Ttest_indResult(statistic=23.894404297284147, pvalue=1.0)

In [6]:
#
# --- Mean Squared Error ---
#
ttest_ind((
    (df_ret - df_ica_pred)**2).unstack().values,
    ((df_ret - df_arma_pred)**2).abs().unstack().values,
    alternative="less")

Ttest_indResult(statistic=2.7116056511512734, pvalue=0.9966503663220891)

In [7]:
#
# --- R (Pearson Correlation) ---
#
pearsonr(df_ret.unstack(), df_ica_pred.unstack(), alternative="greater")

PearsonRResult(statistic=0.025380738748157654, pvalue=0.0005389274663743723)

In [8]:
#
# --- directional accuracy (correct sign of returns was predicted) ---
#
ttest_ind(
    (np.sign(df_ret.unstack()) == np.sign(df_ica_pred.unstack())),
    (np.sign(df_ret.unstack()) == np.sign(df_arma_pred.unstack())),
    alternative="greater")

Ttest_indResult(statistic=1.1754549978778297, pvalue=0.11991058316562926)